In [3]:
import time

from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="My-app")
location = geolocator.geocode("via roma")
print(location.address)

Via Roma, Terracina, Latina, Lazio, 04049, Italia


In [2]:
import pandas as pd

df_private = pd.read_csv("../datasets/synthesized_&_clenaned/strutture_sanitarie_private.csv", delimiter=";")
df_publiche = pd.read_csv("../datasets/cleaned/strutture_sanitare_pubbliche.csv", on_bad_lines="skip", delimiter=";",
                          encoding="utf-8")
df_paraf = pd.read_csv("../datasets/cleaned/parafarmacie.csv", on_bad_lines="skip", delimiter=";", encoding="utf-8")
df_farma = pd.read_csv("../datasets/cleaned/farmacie.csv", on_bad_lines="skip", delimiter=";", encoding="utf-8")
df_private.columns = df_private.columns.str.strip()

In [7]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

regex = r"\d{5}"

geolocator = Nominatim(user_agent="pippobaudofacevalacacca")
geocode = RateLimiter(geolocator.geocode,min_delay_seconds=0.1)
locations = []
position_lat = []
position_long = []
#i = 0
for items in df_private["Indirizzo"] + " " + df_private["Citta\'"]:
    #if(i > 50): break
    location = geocode(items)
    if(location is not None):
        cap = re.search(regex,location.address)
        if cap is not None:
            locations.append(cap.group())
        else:
            locations.append(00000)
        if location.latitude is not None:
            position_lat.append(location.latitude)
        else:
            position_lat.append(0.0)
        if location.longitude is not None:
            position_long.append(location.longitude)
        else:
            position_lat.append(0.0)
    #i += 1
df_private.insert(loc=0, column='CAP', value=pd.Series(locations))
df_private.insert(loc=0, column='LONGITUDINE', value=pd.Series(position_long))
df_private.insert(loc=0, column='LATITUDINE', value=pd.Series(position_lat))


RateLimiter caught an error, retrying (0/2 tries). Called with (*('C.DA DARA 744/D Marsala',), **{}).
Traceback (most recent call last):
  File "C:\Users\marco\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\marco\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\marco\miniconda3\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "C:\Users\marco\miniconda3\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "C:\Users\marco\miniconda3\lib\http\client.py", line 279, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\marco\miniconda3\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\marco\miniconda3\lib\ssl.py", line 1274, in re

In [14]:
from pathlib import Path
filepath = Path('../datasets/out.csv')
df_private.to_csv(filepath)